# Team members
- Bryce Grahn
- Michael Rolle
- Werner de jager
- Abdul Gany Osman
- Lavania Naidoo

# Introduction
The aim of this notebook is undertaking a machine learning investigation on the....

# 1. Imports and installations

In [ ]:
#PIP
# update plotly and pandas_profiling version
!pip install --upgrade plotly
!pip install sweetviz
!pip install autoviz

In [ ]:

# imports
import os
import math
import numpy as np
import scipy as sp
import pandas as pd
import sympy as sym
import seaborn as sns
import sklearn as sk
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import matplotlib.pyplot as plt
import sweetviz as sw

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, log_loss
from sklearn.linear_model import LinearRegression, SGDRegressor, SGDClassifier, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from itertools import chain
from scipy.stats.mstats import winsorize

# define plt settings 
plt.rcParams["font.size"] = 20 
plt.rcParams["axes.labelsize"] = 20 
plt.rcParams["xtick.labelsize"] = 20 
plt.rcParams["ytick.labelsize"] = 20 
plt.rcParams["legend.fontsize"] = 20 
plt.rcParams["figure.figsize"] = (20,10)

# define seaborn settings seaborn
sns.set(style="ticks", color_codes=True)
sns.set_palette("husl")

## 1.1 Import Dataset

In [ ]:
url = '/kaggle/input/payments-hackathon-rojones/Payments Fraud DataSet/'

customers_df = pd.read_csv(url + "customers.csv", index_col = 'CUSTOMER_ID')
terminals_df = pd.read_csv(url + "terminals.csv", index_col = 'TERMINAL_ID')
merchants_df = pd.read_csv(url + "merchants.csv", index_col = 'MERCHANT_ID')
transactions_test_df = pd.read_csv(url + "transactions_test.csv", index_col = 'TX_ID')


# Remove last row in the train dataset to resolve multiple type issue in IS_RECURRING_TRANSACTION column
transactions_train_df = pd.read_csv(url + "transactions_train.csv", index_col = 'TX_ID', dtype={'IS_RECURRING_TRANSACTION': str})
transactions_train_df = transactions_train_df[transactions_train_df['IS_RECURRING_TRANSACTION'] != 'Fals']
transactions_train_copy = transactions_train_df
transactions_train_df['IS_RECURRING_TRANSACTION'] = transactions_train_df['IS_RECURRING_TRANSACTION'].astype(bool)

## display datasets

In [ ]:
print("Customers dataset:")
print("----------------------")
display(customers_df.info())
print("\nterminals dataset:")
print("----------------------")
display(terminals_df.info())
print("\nmerchants dataset:")
print("----------------------")
display(merchants_df.info())
print("\ntransactions_test dataset:")
print("----------------------")
display(transactions_test_df.info())
print("\ntransactions_train dataset:")
print("----------------------")
display(transactions_train_df.info())
print("\ninput dataset:")
print("----------------------")

In [ ]:
print("Customers dataset:")
print("----------------------")
display(customers_df.head())
print("\nterminals dataset:")
print("----------------------")
display(terminals_df.head())
print("\nmerchants dataset:")
print("----------------------")
display(merchants_df.head())
print("\ntransactions_test dataset:")
print("----------------------")
display(transactions_test_df.head())
print("\ntransactions_train dataset:")
print("----------------------")
display(transactions_train_df.head())

## 1.2 Merge datasets


In [ ]:
merged_train_df = pd.merge(transactions_train_df, merchants_df, on='MERCHANT_ID') 
merged_train_df = pd.merge(merged_train_df, customers_df, on='CUSTOMER_ID')
merged_train_df = pd.merge(merged_train_df, terminals_df, on='TERMINAL_ID')

merged_test_df = pd.merge(transactions_test_df, merchants_df, on='MERCHANT_ID')
merged_test_df = pd.merge(merged_test_df, customers_df, on='CUSTOMER_ID')
merged_test_df = pd.merge(merged_test_df, terminals_df, on='TERMINAL_ID')

In [ ]:
# Count the number of null values per column
null_counts_train = merged_train_df.isnull().sum()
null_counts_test = merged_test_df.isnull().sum()

print(f"Number of columns with null data in training set: {len(null_counts_train[null_counts_train > 0].tolist())}")
print(f"Number of columns with null data in test set: {len(null_counts_test[null_counts_test > 0].tolist())}")

# Plot null counts for training set
plt.figure(figsize=(10, 6))
null_counts_train[null_counts_train > 0].plot(kind='bar')
plt.title('Null Counts per Column (Training)')
plt.xlabel('Columns')
plt.ylabel('Null Counts')
plt.show()

# Plot null counts for test set
plt.figure(figsize=(10, 6))
null_counts_test[null_counts_test > 0].plot(kind='bar')
plt.title('Null Counts per Column (Test)')
plt.xlabel('Columns')
plt.ylabel('Null Counts')
plt.show()

In [ ]:
# Calculate the percentage of null values per column
null_percentage_train = (merged_train_df.isnull().sum() / len(merged_train_df)) * 100
null_percentage_test = (merged_test_df.isnull().sum() / len(merged_test_df)) * 100

# Filter to only show columns with null values
null_percentage_train_filtered = null_percentage_train[null_percentage_train > 0]
null_percentage_test_filtered = null_percentage_test[null_percentage_test > 0]

print(f"Number of columns with null data in training set: {len(null_percentage_train_filtered)}")
print(f"Number of columns with null data in test set: {len(null_percentage_test_filtered)}")

# Plot percentage of null values for training set
plt.figure(figsize=(10, 6))
null_percentage_train_filtered.plot(kind='bar')
plt.title('Percentage of Null Values per Column (Training)')
plt.xlabel('Columns')
plt.ylabel('Percentage of Null Values')
plt.show()

# Plot percentage of null values for test set
plt.figure(figsize=(10, 6))
null_percentage_test_filtered.plot(kind='bar')
plt.title('Percentage of Null Values per Column (Test)')
plt.xlabel('Columns')
plt.ylabel('Percentage of Null Values')
plt.show()


## 1.3 Fill null values

In [ ]:
merged_test_df = merged_test_df.fillna('')
merged_train_df = merged_train_df.fillna('')

### Manipulata data


In [ ]:
merged_test_df['DISTANCE_FROM_TERMINAL'] = np.sqrt((merged_test_df.x_customer_id - merged_test_df.x_terminal_id)**2 + (merged_test_df.y_customer_id - merged_test_df.y_terminal__id)**2)['DISTANCE_FROM_TERMINAL'] = np.sqrt((merged_test_df.x_customer_id - merged_test_df.x_terminal_id)**2 + (merged_test_df.y_customer_id - merged_test_df.y_terminal__id)**2)
merged_train_df['DISTANCE_FROM_TERMINAL'] = np.sqrt((merged_train_df.x_customer_id - merged_train_df.x_terminal_id)**2 + (merged_train_df.y_customer_id - merged_train_df.y_terminal__id)**2)['DISTANCE_FROM_TERMINAL'] = np.sqrt((merged_train_df.x_customer_id - merged_train_df.x_terminal_id)**2 + (merged_train_df.y_customer_id - merged_train_df.y_terminal__id)**2)

## visualize data

In [ ]:
# dataset_report = sw.analyze(merged_train_df, "TX_AMOUNT")
# dataset_report.show_notebook(layout='vertical')

## 1.4 Extract y and x datasets

In [ ]:

y_total = merged_train_df[['TX_FRAUD']]
x_total = merged_train_df.drop('TX_FRAUD', axis=1)


## 1.5 Split data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_total, y_total, test_size=0.1, random_state=0)
display(x_total)